# Guide to Using the LastMile Instrumentor for LlamaIndex
This guide will walk you through the process of using the LastMile Instrumentor for LlamaIndex. The LastMile Instrumentor is a powerful tool that allows you to trace and debug your LlamaIndex applications, providing valuable insights into their performance and behavior.



Prerequisites
Before we get started, make sure you have the following prerequisites installed:

Python 3.x
LlamaIndex
OpenAI API key

To install the required dependencies, run the first  cell

In [ ]:
!pip install llama-index-embeddings-openai
!pip install llama-index-embeddings-openai --upgrade

%pip install -q html2text llama-index pandas pyarrow tqdm
%pip install -q llama-index-readers-web
%pip install -q llama-index-callbacks-openinference
!pip install openai --upgrade

# Step 1: Import Required Libraries

First, let's import the necessary libraries for our project.

In [ ]:
import os
from getpass import getpass

import dotenv
import llama_index.core
from lastmile_eval.rag.debugger.tracing.auto_instrumentation import \
    LlamaIndexCallbackHandler

## Step 2: Configure the LastMile Instrumentor

Next, we need to configure the LastMile Instrumentor by setting the global handler for LlamaIndex.

In [2]:
import llama_index.core

from lastmile_eval.rag.debugger.tracing.auto_instrumentation import (
    LlamaIndexCallbackHandler,
)
llama_index.core.global_handler = LlamaIndexCallbackHandler(
    project_name="LlamaIndex Paul Graham QA",
)

2024-05-06 16:20:07,373 - Overriding of current TracerProvider is not allowed
2024-05-06 16:20:07,396 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-06 16:20:07,478 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=my-project HTTP/1.1" 200 325


OpenAI API key configured


# Step 3: Set Up OpenAI API Key

In [ ]:
dotenv.load_dotenv()
if os.getenv("OPENAI_API_KEY") is None:
    os.environ["OPENAI_API_KEY"] = getpass(
        "Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n"
    )
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

# Step 4: Load and Process Documents


In [4]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader().load_data(
    [
        "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
    ]
)

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

2024-05-06 16:20:12,776 - Starting new HTTPS connection (1): raw.githubusercontent.com:443
2024-05-06 16:20:12,866 - https://raw.githubusercontent.com:443 "GET /run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt HTTP/1.1" 200 28419


# Step 5: Create an Index and Query Engine

In [ ]:
from llama_index.embeddings.openai.base import OpenAIEmbedding

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()


# Step 6: Query the Index

In [ ]:
max_characters_per_line = 80
queries = [
    "What did Paul Graham do growing up?",
    "When and how did Paul Graham's mother die?",
    "What, in Paul Graham's opinion, is the most distinctive thing about YC?",
    "When and how did Paul Graham meet Jessica Livingston?",
    "What is Bel, and when and where was it written?",
]
for query in queries:
    response = query_engine.query(query)
    print("Query")
    print("=====")
    print(textwrap.fill(query, max_characters_per_line))
    print()
    print("Response")
    print("========")
    print(textwrap.fill(str(response), max_characters_per_line))
    print()